In [28]:
import requests
from dotenv import load_dotenv
import os

# 환경변수 불러오기
load_dotenv()

# 환경변수에서 키, 엔드포인트 불러오기
endpoint_base = os.getenv("clu_endpoint")  
api_key = os.getenv("clu_key")
api_id = os.getenv("clu_id")
project_name = "PPCLU"
deployment_name = "ppv1-test"


# 요청 헤더
headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
    "Apim-Request-Id" : api_id
}

# 요청 본문 (Query)
query = {
    "kind": "Conversation",
    "analysisInput": {
        "conversationItem": {
            "id": "1",
            "participantId": "user",
            "text": "예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요",
            "modality": "text",
            "language": "ko"
        }
    },
    "parameters": {
        "projectName": project_name,
        "deploymentName": deployment_name,
        "verbose": True,
        "stringIndexType": "TextElement_V8"
    }
}

# 응답 보기 좋게 출력하는 함수
def pretty_print_clu_result(clu_response):
    try:
        result = clu_response['result']
        prediction = result['prediction']
        intent = prediction['topIntent']
        entities = prediction.get('entities', [])

        print(f'당신: {query["analysisInput"]["conversationItem"]["text"]}')
        print(" 추출된 엔터티:")
        for ent in entities:
            category = ent['category']
            text = ent['text']
            score = ent.get("confidenceScore", 0)
            print(f"- {category}: {text} (신뢰도: {score:.2f})")
    except Exception as e:
        print(" 오류 발생:", e)

# API 호출
response = requests.post(
    endpoint_base,
    headers=headers,
    json=query
)

# 결과 저장
if response.status_code == 200:
    clu_response = response.json()  #  저장
    pretty_print_clu_result(clu_response)  # 출력도 계속 하고
else:
    print(f" 요청 실패: {response.status_code}")
    print(response.text)
    clu_response = None  # 실패 시에도 변수 정의

# 클루 결과 객체를 그대로 다음 파이프라인으로 넘기기 위해 출력
clu_response  # Jupyter Notebook에서 마지막 줄에 출력



당신: 예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요
 추출된 엔터티:
- 예산: 3천만 원 (신뢰도: 1.00)
- 지역: 서울 (신뢰도: 1.00)
- 일정: 3월 말 (신뢰도: 1.00)
- 장르: 뮤지컬 (신뢰도: 1.00)


{'kind': 'ConversationResult',
 'result': {'query': '예산은 3천만 원 정도 있고, 서울에서 3월 말쯤 뮤지컬 하나 해보고 싶어요',
  'prediction': {'topIntent': '공연_기획',
   'projectKind': 'Conversation',
   'intents': [{'category': '공연_기획', 'confidenceScore': 1},
    {'category': 'None', 'confidenceScore': 0}],
   'entities': [{'category': '예산',
     'text': '3천만 원',
     'offset': 4,
     'length': 5,
     'confidenceScore': 1},
    {'category': '지역',
     'text': '서울',
     'offset': 17,
     'length': 2,
     'confidenceScore': 1},
    {'category': '일정',
     'text': '3월 말',
     'offset': 22,
     'length': 4,
     'confidenceScore': 1},
    {'category': '장르',
     'text': '뮤지컬',
     'offset': 28,
     'length': 3,
     'confidenceScore': 1}]}}}

In [12]:
import pandas as pd
import numpy as np

np.random.seed(42)

# 1. 기본 feature 생성
df = pd.DataFrame({
    'seat': np.random.randint(300, 2000, size=100),
    'marketing': np.random.randint(1000, 5000, size=100),
    'star_power': np.random.randint(1, 6, size=100)
})

# 2. target 컬럼 계산해서 따로 추가
df['audience'] = (
    df['seat'] * 10 +
    df['marketing'] * 1.5 +
    df['star_power'] * 800 +
    np.random.normal(0, 1000, size=100)
)


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = df[['seat', 'marketing', 'star_power']]
y = df['audience']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [14]:
def predict_audience_rf(seat, marketing, star_power):
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    return int(pred)


In [15]:
seat = 1000
marketing = 2000
star_power = 5

predicted = predict_audience_rf(seat, marketing, star_power)
print(f"🎯 랜덤포레스트 예측 관객 수: {predicted:,}명")

🎯 랜덤포레스트 예측 관객 수: 17,623명


In [16]:
import os
import json
import requests
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
from openai.types.beta.threads import Message

load_dotenv()

def print_pretty_messages(messages):
    print("\n📨 메시지 대화 로그\n" + "-"*40)
    for message in reversed(messages.data):  # 최신순 → 사용자 → GPT 순서로 정렬
        role = "🧑 사용자" if message.role == "user" else "🤖 GPT"
        print(f"\n[{role}]")
        for block in message.content:
            if block.type == "text":
                print(block.text.value)
        print("-"*40)

def make_prediction_prompt(genre, seat, region, marketing, star_power, audience_pred):
    return f"""
안녕하세요! 아래와 같은 정보로 새 공연을 기획 중이시군요.

▶ 장르: {genre}
▶ 지역: {region}
▶ 좌석 수: {seat:,}석
▶ 마케팅 예산: {marketing:,}만 원
▶ 스타 파워: {star_power}점

이에 기반한 예측 모델(Random Forest)을 적용한 결과:

🎯 예상 총 관객 수는 **약 {audience_pred:,}명**으로 추정됩니다.
평일 평균 점유율은 60%, 주말은 85% 수준으로 예측됩니다.

추가로 손익분기점, 예상 수익 분석, 마케팅 전략 추천도 가능합니다.
필요하시면 알려주세요!
"""

client = AzureOpenAI(
  azure_endpoint = os.getenv("oai_assistant_endpoint"),
  api_key= os.getenv("oai_assistant_key"),
  api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
  model="gpt-4o", # replace with model deployment name.
  instructions="""너는 한국의 공연 정보를 잘 알고 있는 똑똑한 문화예술 전문가 도우미야.
사용자가 질문하면, 벡터 저장소에서 관련 정보를 찾아서 친절하고 정확하게 알려줘.
응답은 정중한 말투를 사용하며, 공연/ 예매정보/상세정보/ 통계등을 포함해 요약해줘.""",
  tools=[{"type":"file_search"},{"type":"code_interpreter"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_rk9EOjktnff6RYK12Hx5jn7J"]},"code_interpreter":{"file_ids":[]}},
  temperature=0.3,
  top_p=0.95
)

prompt = make_prediction_prompt(
    genre="뮤지컬",
    seat=1000,
    region="서울",
    marketing=2000,
    star_power=5,
    audience_pred=predict_audience_rf(1000, 2000, 5)
)

def predict_audience_rf(seat, marketing, star_power):
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    return int(pred)


# Create a thread
thread = client.beta.threads.create()

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=input("내용을 입력하세요")
)


# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed':
  messages = client.beta.threads.messages.list(thread_id=thread.id)
  print(print_pretty_messages(messages))
elif run.status == 'requires_action':
  # the assistant requires calling some functions
  # and submit the tool outputs back to the run
  pass
else:
  print(run.status)



📨 메시지 대화 로그
----------------------------------------

[🧑 사용자]
5월 좌석 예측좀 해줘
----------------------------------------

[🤖 GPT]
5월의 공연 좌석 예측을 위해, 관련된 공연 정보와 좌석 수를 분석하여 제공하겠습니다.

1. **대중음악 공연**:
   - **SHINee WORLD Ⅵ: PERFECT ILLUMINATION**: 인스파이어 아레나에서 5월 24일부터 26일까지 진행되며, 좌석 수는 15,000석입니다【4:7†source】.
   - **HIGHLIGHT LIVE: LIGHTS GO ON, AGAIN**: 올림픽공원 KSPO DOME에서 5월 10일부터 12일까지 진행되며, 좌석 수는 15,000석입니다【4:7†source】.

2. **뮤지컬 공연**:
   - **디어 에반 핸슨**: 충무아트센터 대극장에서 3월 28일부터 6월 23일까지 진행되며, 좌석 수는 1,250석입니다【4:7†source】.

3. **클래식 음악 공연**:
   - **서울시향 얍 판 츠베덴과 힐러리 한**: 예술의전당에서 5월 10일 하루 진행됩니다【4:9†source】.
   - **서울스프링실내악축제**: 예술의전당에서 5월 4일과 5일에 각각 진행됩니다【4:4†source】.

4. **연극 공연**:
   - **역사시비 프로젝트, 역사탐험연구소**: 예술공간 혜화에서 5월 3일부터 12일까지 진행되며, 총 8회 상연됩니다【4:17†source】.
   - **제45회 서울연극제**: 예술공간 혜화에서 5월 16일부터 26일까지 진행되며, 총 7회 상연됩니다【4:14†source】.

5. **복합 공연**:
   - **광주상설공연**: 광주공연마루에서 5월 4일부터 26일까지 진행됩니다【4:2†source】.

이 데이터를 바탕으로, 특정 공연의 좌석 점유율을 예측하거나 분석이 필요하면 추가로 요청해주세요!
----------------------------

## 랜덤포레스트 좌석 예측 ##

In [30]:
import os
import json
import requests
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from dotenv import load_dotenv
from openai import AzureOpenAI

# 환경변수 로드
load_dotenv()

# ============================================
# 1. 유틸리티 함수: 대화 로그 프린트 및 GPT 프롬프트 생성
# ============================================
def print_pretty_messages(messages):
    print("\n📨 메시지 대화 로그\n" + "-" * 40)
    for message in reversed(messages.data):  # 최신순으로 출력
        role = "🧑 사용자" if message.role == "user" else "🤖 GPT"
        print(f"\n[{role}]")
        for block in message.content:
            if block.type == "text":
                print(block.text.value)
        print("-" * 40)

def make_prediction_prompt(genre, seat, region, marketing, star_power, audience_pred, weekday_rate, weekend_rate):
    return f"""
안녕하세요! 아래와 같은 정보로 새 공연을 기획 중이시군요.

▶ 장르: {genre}
▶ 지역: {region}
▶ 좌석 수: {seat:,}석
▶ 마케팅 예산: {marketing:,}만 원
▶ 스타 파워: {star_power}점

이에 기반한 예측 모델(Random Forest Regressor)을 적용한 결과:

▶ 예상 관객 수는 약 {audience_pred:,}명 (±500명) 정도로 추정됩니다.
▶ 평일 평균 점유율은 {weekday_rate}%,
   주말 평균 점유율은 {weekend_rate}%로 예상됩니다.

(대시보드 링크 안내)
대시보드에서 일자별 누적 관객 추이 그래프와 유사 공연 대비 비교 바차트를 보실 수 있습니다.

다른 정보도 궁금하신가요? 예를 들어 손익분기점, ROI 분석도 가능합니다.
"""

# ============================================
# 2. 랜덤포레스트 모델 학습 (더미 데이터 기반)
# ============================================
np.random.seed(42)
dummy_data = pd.DataFrame({
    'seat': np.random.randint(300, 2000, size=50),
    'marketing': np.random.randint(1000, 5000, size=50),
    'star_power': np.random.randint(1, 6, size=50)
})
dummy_target = (
    dummy_data['seat'] * 10 +
    dummy_data['marketing'] * 1.5 +
    dummy_data['star_power'] * 800 +
    np.random.normal(0, 1000, size=50)
)
rf_model = RandomForestRegressor(n_estimators=10, random_state=42)
rf_model.fit(dummy_data, dummy_target)

def predict_audience_rf(seat, marketing, star_power):
    """
    입력된 좌석 수, 마케팅 예산(만원), 스타 파워(1~5점)를 기반으로
    예상 관객 수를 예측합니다.
    추가로, 평일 점유율은 60%, 주말 점유율은 85%로 고정하여 반환합니다.
    (실제 환경에서는 이 부분을 더 정교한 모델로 대체할 수 있습니다.)
    """
    input_data = pd.DataFrame([{
        'seat': seat,
        'marketing': marketing,
        'star_power': star_power
    }])
    pred = rf_model.predict(input_data)[0]
    # 예시: 실제 예측값이 약 8500명 정도 나오도록 (±500명 오차 범위)
    # 그리고 평일 60%, 주말 85%로 고정
    weekday_rate = 60
    weekend_rate = 85
    return int(pred), weekday_rate, weekend_rate

# ============================================
# 3. CLU 호출: Azure CLU API에서 사용자 발화를 분석
# ============================================
def call_clu_api(text):
    clu_endpoint_base = os.getenv("clu_endpoint")  # 예: https://language5team.cognitiveservices.azure.com
    clu_api_key = os.getenv("clu_key")
    clu_id = os.getenv("clu_id")
    project_name = "PPCLU"
    deployment_name = "ppv1-test"
    headers = {
        "Ocp-Apim-Subscription-Key": clu_api_key,
        "Content-Type": "application/json",
        "Apim-Request-Id": clu_id
    }
    url = clu_endpoint_base
    body = {
        "kind": "Conversation",
        "analysisInput": {
            "conversationItem": {
                "id": "1",
                "participantId": "user",
                "text": text,
                "modality": "text",
                "language": "ko"
            }
        },
        "parameters": {
            "projectName": project_name,
            "deploymentName": deployment_name,
            "verbose": True,
            "stringIndexType": "TextElement_V8"
        }
    }
    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"CLU API 호출 실패: {response.status_code}")
        print(response.text)
        return None

# (rag_agent_utils.py에 정의된 함수들을 import)
from rag_agent_utils import parse_clu_response, build_kopis_query, make_gpt_prompt

# ============================================
# 4. Azure OpenAI GPT 호출 및 스레드 실행 (RAG)
# ============================================
client = AzureOpenAI(
    azure_endpoint=os.getenv("oai_assistant_endpoint"),
    api_key=os.getenv("oai_assistant_key"),
    api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o",  # 모델 배포 이름 (예: gpt-4o)
    instructions="""너는 한국의 공연 정보를 잘 알고 있는 똑똑한 문화예술 전문가 도우미야.
사용자가 질문하면, 벡터 저장소에서 관련 정보를 찾아서 친절하고 정확하게 알려줘.
응답은 정중한 말투를 사용하며, 공연/예매정보/상세정보/통계 등을 포함해 요약해줘.""",
    tools=[{"type": "file_search"}, {"type": "code_interpreter"}],
    tool_resources={"file_search": {"vector_store_ids": ["vs_rk9EOjktnff6RYK12Hx5jn7J"]}, "code_interpreter": {"file_ids": []}},
    temperature=0.3,
    top_p=0.95
)

def request_gpt(prompt):
    # Azure OpenAI GPT 호출용 URL (전체 URL)
    azure_oai_endpoint = os.getenv("oai_endpoint") 
    azure_oai_key = os.getenv("oai_key")
    headers = {
        "Content-Type": "application/json",
        "api-key": azure_oai_key
    }
    body = {
        "messages": [
            {
                "role": "system",
                "content": "공연 산업에 대한 지식이 아주 해박하여, 관련 질문에 대해 자세히 답변할 수 있는 전문가 역할을 수행해 주세요."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1000
    }
    response = requests.post(azure_oai_endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        citations = response_json['choices'][0]['message'].get('context', {}).get('citations', [])
        return content_text, citations
    else:
        return response.status_code, response.text

# ============================================
# 5. 메인 통합 파이프라인 실행
# ============================================
def main():
    # 사용자 발화 예시 (관객 예측 요청)
    user_query = "이번에 새로 준비 중인 뮤지컬의 관객 수를 예측하고 싶어요."
    print("질문:", user_query)

    # CLU API 호출 → 엔터티 추출
    clu_resp = call_clu_api(user_query)
    if not clu_resp:
        return
    entities = parse_clu_response(clu_resp)
    print("\n추출된 엔터티:", entities)

    # 여기서는 사용자로부터 추가 정보를 수집했다고 가정 (좌석, 마케팅 예산, 스타파워)
    seat = 1000          # 좌석 수
    marketing = 2000     # 마케팅 예산 (만원)
    star_power = 5       # 스타 파워 (1~5점)

    # 관객 예측 (RandomForest 모델)
    audience_pred, weekday_rate, weekend_rate = predict_audience_rf(seat, marketing, star_power)
    print(f"\n예상 관객 수: {audience_pred:,}명")
    print(f"평일 평균 점유율: {weekday_rate}%, 주말 평균 점유율: {weekend_rate}%")

    # KOPIS 검색 조건 생성 (이미 CLU에서 장르, 지역, 일정 등이 추출되었다고 가정)
    kopis_query = build_kopis_query(entities)
    print("\n생성된 KOPIS 검색 조건:", kopis_query)

    # (더미) KOPIS API 호출 → 실제 공연 결과 (여기서는 더미 문자열 사용)
    def call_kopis_api(kopis_query):
        return (f"- 더미 공연 결과: {kopis_query.get('genre', '공연')} 공연 in {kopis_query.get('region', '지역')} "
                f"from {kopis_query.get('startDate', '날짜')} to {kopis_query.get('endDate', '날짜')}.")
    kopis_results = call_kopis_api(kopis_query)
    print("\nKOPIS API 검색 결과:", kopis_results)

    # GPT 프롬프트 생성 (예측 결과와 KOPIS 검색 결과 포함)
    gpt_prompt = make_prediction_prompt(
        genre=entities.get("장르", "뮤지컬"),
        seat=seat,
        region=entities.get("지역", "서울"),
        marketing=marketing,
        star_power=star_power,
        audience_pred=audience_pred,
        weekday_rate=weekday_rate,
        weekend_rate=weekend_rate
    )
    print("\n생성된 GPT 프롬프트:\n", gpt_prompt)

    # GPT 호출하여 최종 응답 생성
    gpt_response, citations = request_gpt(gpt_prompt)
    print("\n🧠 GPT 응답:\n", gpt_response)
    print("\n📚 출처:\n", citations)

    # Azure OpenAI 챗봇 스레드 실행 (실제 대화)
    thread = client.beta.threads.create()
    user_input = input("\n대화 입력 (내용을 입력하세요): ")
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    while run.status in ['queued', 'in_progress', 'cancelling']:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
    if run.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print_pretty_messages(messages)
    elif run.status == 'requires_action':
        print("추가 작업 필요")
    else:
        print("스레드 상태:", run.status)

if __name__ == "__main__":
    main()


질문: 이번에 새로 준비 중인 뮤지컬의 관객 수를 예측하고 싶어요.

추출된 엔터티: {'장르': '뮤지컬'}

예상 관객 수: 16,614명
평일 평균 점유율: 60%, 주말 평균 점유율: 85%

생성된 KOPIS 검색 조건: {'genre': '뮤지컬', 'region': '', 'startDate': '2025-04-07', 'endDate': '2025-04-17'}

KOPIS API 검색 결과: - 더미 공연 결과: 뮤지컬 공연 in  from 2025-04-07 to 2025-04-17.

생성된 GPT 프롬프트:
 
안녕하세요! 아래와 같은 정보로 새 공연을 기획 중이시군요.

▶ 장르: 뮤지컬
▶ 지역: 서울
▶ 좌석 수: 1,000석
▶ 마케팅 예산: 2,000만 원
▶ 스타 파워: 5점

이에 기반한 예측 모델(Random Forest Regressor)을 적용한 결과:

▶ 예상 관객 수는 약 16,614명 (±500명) 정도로 추정됩니다.
▶ 평일 평균 점유율은 60%,
   주말 평균 점유율은 85%로 예상됩니다.

(대시보드 링크 안내)
대시보드에서 일자별 누적 관객 추이 그래프와 유사 공연 대비 비교 바차트를 보실 수 있습니다.

다른 정보도 궁금하신가요? 예를 들어 손익분기점, ROI 분석도 가능합니다.



ConnectionError: HTTPSConnectionPool(host='oai-teamaim.openai.azure.com', port=443): Max retries exceeded with url: /openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000021BFF0583A0>: Failed to resolve 'oai-teamaim.openai.azure.com' ([Errno 11001] getaddrinfo failed)"))